In [14]:
import ee
import geemap
import xarray as xr
import xee

# احراز هویت و مقداردهی اولیه
ee.Authenticate()
ee.Initialize(project='ee-pwuyaz', opt_url='https://earthengine-highvolume.googleapis.com')

# ایجاد نقشه
Map = geemap.Map(basemap='SATELLITE')
Map

# تعریف منطقه مورد نظر (خلیج فارس)
persian_gulf = ee.Geometry.Rectangle([47, 23, 56, 31])

# مرکز کردن نقشه روی منطقه مورد نظر
Map.centerObject(persian_gulf, 6)

# دریافت داده‌های جریان اقیانوسی
ocean_currents = (ee.ImageCollection('HYCOM/sea_water_velocity')
                  .filterDate('2000-01-01', '2025-12-31')
                  .select(['velocity_u_0', 'velocity_v_0']))

# لیست سال‌ها و ماه‌ها
years = ee.List.sequence(2000, 2025)
months = ee.List.sequence(1, 12)

# تابع برای تولید ویژگی‌های ماهانه
def create_monthly_features(year):
    year = ee.Number(year)
    
    def create_features_for_month(month):
        month = ee.Number(month)
        start_date = ee.Date.fromYMD(year, month, 1)
        end_date = start_date.advance(1, 'month')

        # محاسبه میانگین برای داده‌های جریان اقیانوسی در ماه
        currents = ocean_currents.filterDate(start_date, end_date).mean().rename(['velocity_u', 'velocity_v'])

        # افزودن باندهای طول و عرض جغرافیایی
        lon = ee.Image.pixelLonLat().select('longitude').rename('longitude').toFloat()
        lat = ee.Image.pixelLonLat().select('latitude').rename('latitude').toFloat()

        # ترکیب باندها در یک تصویر
        image_with_coords = ee.Image.cat([currents, lon, lat])

        # نمونه‌برداری از تصویر برای دریافت نقاط
        points = image_with_coords.sample(
            region=persian_gulf,
            scale=4000,
            numPixels=1e5,
            seed= 42,
            geometries=True  # کاهش تعداد پیکسل‌ها برای مدیریت حجم داده
        ).limit(1221)

        # افزودن valid_time به هر ویژگی
        def add_valid_time(feature):
            return feature.set('valid_time', start_date.format('M/d/YYYY'))

        return points.map(add_valid_time)

    # تولید ویژگی‌های ماهانه برای هر سال
    monthly_features = months.map(create_features_for_month)
    return ee.FeatureCollection(monthly_features).flatten()

# تولید ویژگی‌ها برای تمام سال‌ها
yearly_features = years.map(create_monthly_features)

# flatten کردن تمام ویژگی‌ها به یک مجموعه ویژگی
all_features = ee.FeatureCollection(yearly_features).flatten()

# صادرات به CSV
ee.batch.Export.table.toDrive(
    collection=all_features,
    description='2000_2025_ocean_currents',
    folder='GEE_Exports',
    fileFormat='CSV',
    selectors=['valid_time', 'latitude', 'longitude', 'velocity_u', 'velocity_v']
).start()

2.

In [18]:


# دریافت داده‌های جریان اقیانوسی
ocean_currents = (ee.ImageCollection('HYCOM/sea_water_velocity')
                  .filterDate('2000-01-01', '2025-12-31')
                  .select(['velocity_u_0', 'velocity_v_0']))

# بررسی وجود داده‌ها
def check_data_availability(start_date, end_date):
    count = ocean_currents.filterDate(start_date, end_date).size()
    return count

# لیست سال‌ها و ماه‌ها
years = ee.List.sequence(2000, 2025)
months = ee.List.sequence(1, 12)

# تابع برای تولید ویژگی‌های ماهانه (برای فایل اصلی)
def create_monthly_features(year):
    year = ee.Number(year)
    
    def create_features_for_month(month):
        month = ee.Number(month)
        start_date = ee.Date.fromYMD(year, month, 1)
        end_date = start_date.advance(1, 'month')

        # بررسی تعداد تصاویر در بازه زمانی
        data_count = check_data_availability(start_date, end_date)
        def handle_data(image_collection):
            currents = image_collection.mean().rename(['velocity_u', 'velocity_v'])
            lon = ee.Image.pixelLonLat().select('longitude').rename('longitude').toFloat()
            lat = ee.Image.pixelLonLat().select('latitude').rename('latitude').toFloat()
            image_with_coords = ee.Image.cat([currents, lon, lat])

            points = image_with_coords.sample(
                region=persian_gulf,
                scale=4000,
                numPixels=1e5,
                seed=42,
                geometries=True
            ).limit(1221)

            def add_valid_time(feature):
                return feature.set('valid_time', start_date.format('M/d/YYYY'))

            return points.map(add_valid_time)

        # فقط در صورتی ادامه بده که داده‌ای وجود داشته باشد
        return ee.Algorithms.If(
            data_count.gt(0),
            handle_data(ocean_currents.filterDate(start_date, end_date)),
            ee.FeatureCollection([])
        )

    monthly_features = months.map(create_features_for_month)
    return ee.FeatureCollection(monthly_features).flatten()

# تابع برای محاسبه میانگین ماهانه در کل خلیج فارس
def create_monthly_mean(year):
    year = ee.Number(year)
    
    def monthly_mean(month):
        month = ee.Number(month)
        start_date = ee.Date.fromYMD(year, month, 1)
        end_date = start_date.advance(1, 'month')

        # بررسی تعداد تصاویر
        data_count = check_data_availability(start_date, end_date)
        
        def compute_mean(image_collection):
            currents = image_collection.mean().rename(['velocity_u', 'velocity_v'])
            
            # محاسبه میانگین در کل منطقه
            mean_values = currents.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=persian_gulf,
                scale=4000,
                maxPixels=1e10
            )

            # بررسی وجود کلیدها
            velocity_u = ee.Algorithms.If(
                mean_values.contains('velocity_u'),
                mean_values.get('velocity_u'),
                None
            )
            velocity_v = ee.Algorithms.If(
                mean_values.contains('velocity_v'),
                mean_values.get('velocity_v'),
                None
            )

            return ee.Feature(None, {
                'valid_time': start_date.format('M/d/YYYY'),
                'mean_velocity_u': velocity_u,
                'mean_velocity_v': velocity_v
            })

        # فقط در صورتی محاسبه کن که داده‌ای وجود داشته باشد
        return ee.Algorithms.If(
            data_count.gt(0),
            compute_mean(ocean_currents.filterDate(start_date, end_date)),
            ee.Feature(None, {
                'valid_time': start_date.format('M/d/YYYY'),
                'mean_velocity_u': None,
                'mean_velocity_v': None
            })
        )

    monthly_means = months.map(monthly_mean)
    return ee.FeatureCollection(monthly_means)

# تولید ویژگی‌ها برای تمام سال‌ها (فایل اصلی)
yearly_features = years.map(create_monthly_features)
all_features = ee.FeatureCollection(yearly_features).flatten()

# تولید میانگین‌های ماهانه برای کل خلیج فارس
yearly_means = years.map(create_monthly_mean)
all_means = ee.FeatureCollection(yearly_means).flatten()

# صادرات فایل اصلی به CSV
ee.batch.Export.table.toDrive(
    collection=all_features,
    description='2000_2025_ocean_currents',
    folder='GEE_Exports',
    fileFormat='CSV',
    selectors=['valid_time', 'latitude', 'longitude', 'velocity_u', 'velocity_v']
).start()

# صادرات میانگین ماهانه به فایل جداگانه
ee.batch.Export.table.toDrive(
    collection=all_means,
    description='2000_2025_monthly_mean_ocean_currents',
    folder='GEE_Exports',
    fileFormat='CSV',
    selectors=['valid_time', 'mean_velocity_u', 'mean_velocity_v']
).start()